In [3]:
#Setup our Environment ie curves etc
%run InitialSetup.ipynb

'''Data Available

1) Swap Curves based on today
2) Historical Swap Data
3) Historical Black Volatility Data
4) Historical OIS Data
5) Pricers and Hagan Approximation'''

'Data Available\n\n1) Swap Curves based on today\n2) Historical Swap Data\n3) Historical Black Volatility Data\n4) Historical OIS Data\n5) Pricers and Hagan Approximation'

In [4]:
# Construct Surface between 5X5 and 20x20 MaturityXTenor tuples
it_range = [i for i in range(5, 11)] + [15, 20]

# Create tuples of dates
iter_list = list(itertools.product(*[it_range, it_range]))

# Create similar names to the titles of the vol_data file
vol_filt = [str(i)+'Y '+str(j)+'Y' for i, j in iter_list]

# Filter vol_data based on criteria
vol_surface = vol_data[vol_filt]

# Extract the list of dates
calc_dates = [datetime.strftime(date, '%Y-%m-%d') for date in vol_data.index]

In [5]:
# Vectorize functions
Hagan_Vect = np.vectorize(sigma_CEV_Hagan)
Black_Vect = np.vectorize(payer_Black)
Chi_Square_Vect = np.vectorize(payer_chi_square)

In [ ]:
# Construct our curves for each date
#Weird loop done for computational purposes
curves = []
for i in range(1, 590):
    curves.append([swap_curve_construct(x) for x in calc_dates[(i-1)*4:i*4]])

# Flatten the list of lists
curves =  list(itertools.chain.from_iterable(curves))

In [162]:
st = time.tome
test = curves[0]
date  = calc_dates[0]
e_date = DateParser.parseFormatted(date, '%Y-%m-%d')
Settings.instance().setEvaluationDate(e_date)
test.forwardRate(e_date, Date(26, 1, 2021), Thirty360(), Simple).rate()

0.04133909923340138

In [ ]:
#Compute forward swap rates for each of the dates
forward_swap_rates = []

for curve in discount_curves:
    forward_rates.append([curve.forwardRate(eval_date + Period(i, Years), eval_date + Period(i+j, Years), 
                                            Thirty360(), Simple).rate() for i, j in prange(iter_list)])

In [ ]:
# We combine all our data into a single file
# Flatten forward_rates
f_rates = np.array(list(itertools.chain.from_iterable(forward_rates)))

# Flatten Black volatility
v_data = vol_data.to_numpy.flatten()

# Define list of maturities and tenors
maturity_array = np.array([item[1] for item in iter_list]*len(vol_data))
tenor_array = np.array([item[0] for item in iter_list]*len(vol_data))

# Join f_rates, maturity and tenor and sigma
full_data = np.concatenate([maturity_array, tenor_array, f_rates, v_data], axis = 1)

In [ ]:
# Optimization techniques
# Define a bivariate function
calib_func = lambda beta, sigma: np.sum(np.power(Chi_Square_Vect() - full_data[:,3], 2))

# Minimization schemes
